# Practical 1: Fundamentals of Remote Sensing and Digital Image Processing

In this practical, we'll exercise the fundamentals of remote sensing (radiation, transmission, etc). We'll also learn how to do some image processing using Python's [scikit-image](https://scikit-image.org) library.


---

## Loading an image with scikit-image

Images are represented computationally as **2D arrays of integers**.


### Questions:

1. What is the size of the array (number of rows and columns)?
1. What is the data type of the elements in this array?
1. How many bits are required to store each element?
1. How many possible values can be stored?


---

## Plotting images with matplotlib

We can map each possible value in the array to a color using a **colormap**. There are [many types of colormaps](https://matplotlib.org/gallery/color/colormap_reference.html#sphx-glr-gallery-color-colormap-reference-py). The resulting image is a **pseudo-color plot**.


### Task: Select a 500 x 500 pixel region of this image and plot it.

You can choose the region and the colormap. Paste the image you produce into the shared Google Docs.



----

## Image enhancements

Contrast enhancement techniques for grayscale (single band) images:

1. Contrast stretching: increases the contrast of the image by mapping the intensity values of the input image to a new range of values (usually shorter than the original). Function: [skimage.exposure.rescale_intensity](https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.rescale_intensity)
2. Histogram equalization: enhances the contrast of images by transforming the intensity values of an image so that the histogram of the output image approximately matches a specified distribution (for example, a uniform distribution). Function: [skimage.exposure.equalize_hist](https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.equalize_hist)
3. Adaptive histogram equalization: operates on small data regions (tiles) rather than the entire image. Each tile's contrast is enhanced so that the histogram of each output region approximately matches the specified distribution. Function: [skimage.exposure.equalize_adapthist](https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.equalize_adapthist)

These enhancements are used to aid the *visualization* of images. **Be careful to always preserve the original and communicate that you have processed the images.** 

**The enhanced versions are usually not suitable for quantitative analysis (why is that?).**


### Contrast stretching 

Stretches the values of the image to a new interval defined by us. Can be used to limit the effect of very bright or very dim pixels.

### Task: Apply histogram equalization (normal and adaptive) to the image.

Make figures to compare the results. Paste the image you produce into the shared Google Docs. 